In [1]:
# Music Classification - CSCI 4622 Final Project
# Giovanni Visco, Ishan Kumar, Avnish Asthana, Dhruv Patel

# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.utils import shuffle


plt.style.use('ggplot')
import librosa
from librosa import feature
from librosa import display
genres = [
    'blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop',
    'reggae', 'rock'
]

In [2]:
# list data
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/blues\blues.00000.au
./data/blues\blues.00001.au
./data/blues\blues.00002.au
./data/blues\blues.00003.au
./data/blues\blues.00004.au
./data/blues\blues.00005.au
./data/blues\blues.00006.au
./data/blues\blues.00007.au
./data/blues\blues.00008.au
./data/blues\blues.00009.au
./data/blues\blues.00010.au
./data/blues\blues.00011.au
./data/blues\blues.00012.au
./data/blues\blues.00013.au
./data/blues\blues.00014.au
./data/blues\blues.00015.au
./data/blues\blues.00016.au
./data/blues\blues.00017.au
./data/blues\blues.00018.au
./data/blues\blues.00019.au
./data/blues\blues.00020.au
./data/blues\blues.00021.au
./data/blues\blues.00022.au
./data/blues\blues.00023.au
./data/blues\blues.00024.au
./data/blues\blues.00025.au
./data/blues\blues.00026.au
./data/blues\blues.00027.au
./data/blues\blues.00028.au
./data/blues\blues.00029.au
./data/blues\blues.00030.au
./data/blues\blues.00031.au
./data/blues\blues.00032.au
./data/blues\blues.00033.au
./data/blues\blues.00034.au
./data/blues\blues.0

In [3]:
# create a list of all features

features = 'filename chroma_stft_mean chroma_stft_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var tempo' 
for i in range(1, 21):
    features += f' mfcc{i}_mean'
    features += f' mfcc{i}_var'
    
features += ' genre'
features = features.split()

# create an empty dataframe
dfFeatures = pd.DataFrame(columns = features)
dfFeatures

,filename,chroma_stft_mean,chroma_stft_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,genre


In [4]:
# load audio time series into a list using librosa.load, create list of corresponding labels
for genre in genres:
    # identify the path using the genre
    path = './data/' + genre
    # loop through every file in the path
    for file in os.listdir(path):
        filename = path + '/' + file
        
        # load time series and sampling rate
        y, sr = librosa.load(filename, mono = True, duration = 10)
        
        # extract all features from time series and sampling rate
        chromaStft = librosa.feature.chroma_stft(y=y, sr=sr)
        specCent = librosa.feature.spectral_centroid(y=y, sr=sr)
        specBandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zeroCrossing = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        tempo = librosa.beat.tempo(y=y, sr=sr)
        
        # personal check to make sure we dont get dynamic tempo
        if len(tempo) != 1:
            raise ValueError("Dynamic tempo was returned.")
        
        # dictionary to upload current row
        uploadDict = {}
        uploadDict['filename'] = file
        uploadDict['genre'] = genre
        
        # get the mean of all the mfcc
        mfccMeans = []
        for index, value in enumerate(mfcc):
            currMfccMean = f'mfcc{index+1}_mean'
            currMfccVar = f'mfcc{index+1}_var'
            mfccMean = np.mean(value)
            mfccVar = np.var(value)
            uploadDict[currMfccMean] = mfccMean
            uploadDict[currMfccVar] = mfccVar
            

        # get the means and add to dict
        uploadDict['chroma_stft_mean'] = np.mean(chromaStft)
        uploadDict['chroma_stft_var'] = np.var(chromaStft)
        uploadDict['spectral_centroid_mean'] = np.mean(specCent)
        uploadDict['spectral_centroid_var'] = np.var(specCent)
        uploadDict['spectral_bandwidth_mean'] = np.mean(specBandwidth)
        uploadDict['spectral_bandwidth_var'] = np.var(specBandwidth)
        uploadDict['rolloff_mean'] = np.mean(rolloff)
        uploadDict['rolloff_var'] = np.var(rolloff)
        uploadDict['zero_crossing_rate_mean'] = np.mean(zeroCrossing)
        uploadDict['zero_crossing_rate_var'] = np.var(zeroCrossing)
        uploadDict['tempo'] = tempo[0]
        
        # append all values to dataframe
        dfFeatures = dfFeatures.append(uploadDict, ignore_index = True)
            
            
            
        
dfFeatures.head()

,filename,chroma_stft_mean,chroma_stft_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,genre
0,blues.00000.au,0.342053,0.089037,1782.622682,116977.082472,2020.556328,84328.680518,3845.075667,8.328459e+05,0.080111,...,69.972038,-2.870996,35.479565,0.306077,43.506840,-2.700369,58.693989,2.370232,48.047718,blues
1,blues.00001.au,0.358000,0.090658,1364.273766,291346.764439,1842.030467,221553.848927,2976.303348,2.452152e+06,0.055211,...,62.798424,0.497853,61.236259,0.039326,48.482971,-1.083576,54.454639,-0.112309,53.948948,blues
2,blues.00002.au,0.382027,0.087238,1521.556846,224961.620877,1736.799388,64062.188124,3028.062974,8.508550e+05,0.070182,...,37.905025,-6.660074,53.707203,-2.039234,48.657555,-2.045270,45.731483,-2.697336,28.559635,blues
3,blues.00003.au,0.391765,0.094778,947.018874,135762.858041,1427.836298,122599.100317,1863.946038,9.467861e+05,0.030092,...,33.782215,0.231585,31.175877,2.437521,30.588255,1.284240,26.782133,-2.200283,31.076508,blues
4,blues.00004.au,0.316456,0.090731,2046.427867,540932.742010,1858.463045,111903.623326,4005.025898,2.115368e+06,0.116385,...,129.746536,-8.198335,71.464584,-1.059961,62.674892,-3.900608,59.124683,-12.995133,57.077927,blues


In [5]:
dfFeatures.to_csv('music_features_updated.csv', index = False)